# Model Training

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv("data/gemstone.csv")

In [8]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [10]:
df = df.drop(labels=["id"],axis=1)

In [12]:
# Independent and Dependent Feature

X = df.drop(labels=["price"],axis=1)    # -> Independent Feature
Y = df[["price"]]                       # -> Dependent Feature

In [29]:
# Define which columns should be ordinal-encoded and which should be scaled

categorical_cols  = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [23]:
# Define the custom ranking for each ordinal variable

cut_categories = ["Fair","Good","Very Good","Premium","Ideal"]
color_categories = ["D","E","F","G","H","I","J"]
calarity_categories = ["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]


In [21]:
# Handling the missing values in the dataset by using machine learnig technique
from sklearn.impute import SimpleImputer

# handling the Feature Scaling
from sklearn.preprocessing import StandardScaler

# Handling the Oridnal Encoding
from sklearn.preprocessing import OrdinalEncoder


# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [39]:
# Numerical_Pipeline

num_pipline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scalar',StandardScaler())
    ]
    
)


# Categorical_Pipline

cat_pipline = Pipeline(
   steps=[
       ('imputer',SimpleImputer(strategy='most_frequent')),
       ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,calarity_categories])),
       ('salar',StandardScaler())
       ] 
    
)


preprocessor = ColumnTransformer([
    ('num_pipline',num_pipline,numerical_cols),
    ('cat_pipline',cat_pipline,categorical_cols)
    
])

In [37]:
# Train Test Split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=0.30,random_state=42)

In [42]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test =  pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [43]:
X_train.head()

,num_pipline__carat,num_pipline__depth,num_pipline__table,num_pipline__x,num_pipline__y,num_pipline__z,cat_pipline__cut,cat_pipline__color,cat_pipline__clarity
0,-0.629400,-0.771576,-1.163422,-0.519931,-0.500391,-0.568763,0.875600,0.286369,0.015505
1,-0.199354,0.815770,-0.115636,-0.061740,-0.039207,0.018501,0.875600,0.286369,2.677493
2,1.542331,-0.864949,-0.115636,1.582360,1.498073,1.393560,-1.139937,-1.561311,-1.315488
3,0.230692,0.255530,0.408257,0.387468,0.440063,0.433884,-1.139937,0.902263,0.681002
4,-0.908929,-0.024590,0.408257,-0.996091,-0.988704,-0.998469,-0.132169,-0.945418,-0.649991


**Model Training**

In [45]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [47]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [49]:
# How many Cofficient we get
regression.coef_

array([[ 6387.29214721,  -127.14082977,   -58.79037826, -2072.8673303 ,
         -124.41768469,   -30.24011054,    73.69127661,  -462.80288204,
          648.42944134]])

In [50]:
# How many intercept we get

regression.intercept_

array([3978.62111898])

In [52]:
import numpy as np

def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    
    return mae , rmse , r2_square
    

In [58]:
## Training the Models

models = {
    'LinearRegression' : LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'ElasticNet' : ElasticNet()
    
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    
    # Make Prediction
    y_pred = model.predict(X_test)
    mae , rmse , r2_square = evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    
    r2_list.append(r2_square)
    
    print('='*35)
    print('/n')
    

LinearRegression
Model Training Performance
RMSE: 1012.7038347681915
MAE: 675.2075121569702
R2 score 93.69073316627981
/n
Lasso
Model Training Performance
RMSE: 1013.0310558654558
MAE: 676.469713047517
R2 score 93.68665525389078
/n
Ridge
Model Training Performance
RMSE: 1012.7031126091389
MAE: 675.2660398328608
R2 score 93.6907421645523
/n
ElasticNet
Model Training Performance
RMSE: 1540.0211634666093
MAE: 1063.872505293151
R2 score 85.40959442549227
/n
